In [48]:
import re

from abc import abstractmethod

class Instruction():
    
    def __init__(self, value):
        self.value = value
    
    @abstractmethod
    def apply(self, memory, mask):
        pass
    
class MemWriteInstruction(Instruction):
    
    def __init__(self, value):
        super(MemWriteInstruction, self).__init__(value)
        self.memoryaddr = int(self.value[0])
        self.writeval = int(self.value[1])
        
    def apply(self, memory, mask):
        valuestring = "0b"
        for idx, bitstring in enumerate(mask):
            if bitstring == 'X':
                valuestring += str((self.writeval & (1 << (len(mask) - idx - 1))) >> (len(mask) - idx - 1))
            else:
                valuestring += bitstring
        value = int(valuestring, 2)
        memory[self.memoryaddr] = value
        return memory, mask
    
class MaskChangeInstruction(Instruction):
    
    def __init__(self, value):
        super(MaskChangeInstruction, self).__init__(value)
        self.mask = self.value
        
    def apply(self, memory, mask):
        return memory, self.mask

class Computer():
    
    def __init__(self):
        self.memory = dict()
        self.mask = None
    
    def _apply(self, instruction):
        self.memory, self.mask = instruction.apply(self.memory, self.mask)
        
    def run(self, instructionset):
        for instruction in instructionset:
            self._apply(instruction)
        return self.memory, self.mask

maskregex = r"mask=(?P<mask>[X|1|0]{36})"
memregex = r"mem\[(?P<addr>\d+)\]=(?P<val>\d+)"

    
def build(stringrep):
    if re.match(maskregex, stringrep):
        mask = re.match(maskregex, stringrep).group("mask")
        return MaskChangeInstruction(mask)
    elif re.match(memregex, stringrep):
        memaddr = re.match(memregex, stringrep).group("addr")
        memval = re.match(memregex, stringrep).group("val")
        return MemWriteInstruction((memaddr, memval))

filename = "./input.txt"
with open(filename) as f:
    content = [line.strip() for line in f.readlines()]
    content = ["".join(line.split()) for line in content]

computer = Computer()
instructionset = [build(rep) for rep in content]
memory, _ = computer.run(instructionset)
sum(memory.values())

165